In [ ]:
!pip install mediapipe

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from google.colab import drive
import tensorflow as tf
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

In [ ]:
def get_hand_keypoints_2d(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
        return np.array(keypoints).flatten()
    else:
        return None

In [ ]:
def get_hand_keypoints_2d(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error loading image: {image_path}")
        return None

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        keypoints = []
        for hand_landmarks in results.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y])
        return np.array(keypoints).flatten()
    else:
        return None

In [ ]:
#zapisywanie keypointów ze zdjęć, nie trzeba tego odpalać
data_dir = "/content/drive/MyDrive/ED/asl_alphabet_train"
output_dir = "/content/drive/MyDrive/ED/keypoints2d"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for label, folder in enumerate(os.listdir(data_dir)):
    print(label, folder)
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            keypoints = get_hand_keypoints_2d(image_path)
            if keypoints is not None:
                output_file = os.path.join(output_dir, f"{folder}_{image_name.split('.')[0]}.npy")
                np.save(output_file, keypoints)


In [ ]:
keypoints_dir = "/content/drive/MyDrive/ED/keypoints"
keypoints_data = []
keypoints_labels = []

for file_name in os.listdir(keypoints_dir):
    print(file_name)
    file_path = os.path.join(keypoints_dir, file_name)
    keypoints = np.load(file_path)
    label = file_name[0]  # Get the first letter of the file name as the label
    
    keypoints_data.append(keypoints)
    keypoints_labels.append(label)

keypoints_data = np.array(keypoints_data)
keypoints_labels = np.array(keypoints_labels)

In [ ]:
unique_labels = sorted(list(set(keypoints_labels)))
label_to_int = {label: i for i, label in enumerate(unique_labels)}

In [ ]:
keypoints_labels_int = np.array([label_to_int[label] for label in keypoints_labels])

In [ ]:
#model do keypointów 3d (mozliwe ze nie dziala za dobrze xd)
num_classes = len(unique_labels)
input_dim = 21 * 3

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
epochs = 25
history = model.fit(keypoints_data, keypoints_labels_int, epochs = epochs)

In [ ]:
model.save('/content/drive/MyDrive/ED/trained_model2')

In [ ]:
model_path = "/content/drive/MyDrive/ED/trained_model"
model = tf.keras.models.load_model(model_path)